In [3]:
pip install transformers datasets evaluate requests beautifulsoup4 huggingface-hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
conda install tf-keras

ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.

In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
from bs4 import BeautifulSoup
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling,Trainer, TrainingArguments

In [6]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [7]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [8]:
list_of_url=[
    "http://www.knowledgebank.irri.org/decision-tools/rice-doctor/rice-doctor-fact-sheets/item/bacterial-blight",
    "http://www.knowledgebank.irri.org/training/fact-sheets/pest-management/diseases/item/blast-leaf-collar",
    "http://www.knowledgebank.irri.org/training/fact-sheets/pest-management/diseases/item/brown-spot",
    "http://www.knowledgebank.irri.org/training/fact-sheets/pest-management/diseases/item/tungro"
]

In [ ]:
def scrap_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content,'html.parser')
        para = soup.find_all('p')
        content = [para.get_text() for para in para]
        return '\n'.join(content)
    else:
        return ''

In [ ]:
scrap_contents = [scrap_content(url) for url in list_of_url]

In [ ]:
def clean_text_data(text):
    cleaned_text=text.replace(u'\xa0',u'')
    return cleaned_text

In [ ]:
for i in range(len(scrap_contents)):
    clean_data=clean_text_data(scrap_contents[i])
    scrap_contents[i]=clean_data

In [ ]:
scrap_contents

In [ ]:
f=open('rice_disease_info_dataset.txt','w', encoding='utf-8')
for content in scrap_contents:
    f.write(content)

In [9]:
# Function to load and tokenize dataset
def load_dataset(file_path,tokenizer):
    with open(file_path,'r',encoding='utf-8') as f:
        text = f.read()
    encodings = tokenizer(text,return_tensors='pt',max_length=512,truncation=True,padding=True)
    return TextDataset(tokenizer=tokenizer,file_path=file_path,block_size=128)


In [10]:
# Load dataset
filepath="rice_disease_info_dataset.txt"
dataset=load_dataset(file_path=filepath, tokenizer=tokenizer)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [11]:
# Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [14]:
trainer.train()

  0%|          | 0/30 [00:00<?, ?it/s]

: 